In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features, reduce_mem_usage
from preprocessing import get_dummies
import datetime

from tqdm import tqdm
import time
import sys
from joblib import Parallel, delayed

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

In [2]:
#========================================================================
# Classifier
# clf = utils.read_pkl_gzip('../stack/0207_224_outlier_classify_9seed_lgb_binary_CV0-9099420278047783_235features.gz')[[key, 'pred_mean']].set_index(key)
# clf_2 = utils.read_pkl_gzip('../stack/0207_212_outlier_classify_9seed_lgb_binary_CV0-9084737642836664_235features.gz')[[key, 'pred_mean']].set_index(key)
# clf['pred_mean_2'] = clf_2['pred_mean']
# clf['clf_pred'] =  clf['pred_mean'].values*0.9 + clf['pred_mean_2'].values*0.1
# base['clf_pred'] = clf['clf_pred']
base = utils.read_pkl_gzip('../input/base_clf.gz')
#========================================================================

In [29]:
tmp = base[[target, 'clf_pred', 'no_out_flg']]
tmp['no_out_flg'] = tmp['no_out_flg'].fillna(0)
# tmp = tmp[tmp['no_out_flg']==0]
tmp = tmp[tmp[target]<-30]
thres_cnt = tmp['clf_pred'].map(lambda x: np.round(x, 10)).value_counts().sort_index()
# thres_cnt
thres_cnt = thres_cnt.reset_index()
thres_cnt[thres_cnt['index']<0.00048]

,index,clf_pred


In [36]:
# base.loc[base['clf_pred']<0.00048, 'no_out_flg'] = 1
# base['no_out_flg'] = base['no_out_flg'].fillna(0)
# base['no_out_flg'].sum()
utils.to_pkl_gzip(obj=base, path='../input/base_no_out_clf')

In [35]:
base

,card_id,target,first_active_month,hist_purchase_date_max,hist_purchase_month_max,hist_purchase_date_min,hist_purchase_month_min,new_purchase_date_max,new_purchase_month_max,new_purchase_date_min,new_purchase_month_min,hist_personal_term,new_personal_term,hist_regist_term,new_regist_term,no_out_flg,clf_pred
0,C_ID_92a2005557,-0.820283,2017-06-01,2018-02-25 09:31:15,2018-03-01,2017-06-27 14:18:08,2017-06-01,2018-04-29 11:23:05,2018-05-01,2018-03-05 14:04:36,2018-03-01,9,2.0,9,11.0,1.0,0.000444
1,C_ID_3d0044924f,0.392913,2017-01-01,2018-01-31 22:31:09,2018-02-01,2017-01-06 16:29:42,2017-01-01,2018-03-30 06:48:26,2018-04-01,2018-02-01 17:07:54,2018-02-01,13,2.0,13,15.0,0.0,0.007831
2,C_ID_d639edf6cd,0.688056,2016-08-01,2018-02-27 19:08:25,2018-03-01,2017-01-11 08:21:22,2017-01-01,2018-04-28 17:43:11,2018-05-01,2018-04-28 17:43:11,2018-04-01,14,1.0,19,18.0,0.0,0.004074
3,C_ID_186d6a6901,0.142495,2017-09-01,2018-02-28 11:44:40,2018-03-01,2017-09-26 16:22:21,2017-09-01,2018-04-18 11:00:11,2018-05-01,2018-03-07 11:55:06,2018-03-01,6,2.0,6,8.0,0.0,0.000797
4,C_ID_cdbd2c0db2,-0.159749,2017-11-01,2018-02-28 20:40:41,2018-03-01,2017-11-12 00:00:00,2017-11-01,2018-04-28 18:50:25,2018-05-01,2018-03-02 11:55:43,2018-03-01,4,2.0,4,6.0,1.0,0.000251
5,C_ID_0894217f2f,0.871585,2016-09-01,2017-07-29 15:37:18,2017-08-01,2017-01-10 17:14:53,2017-01-01,2017-09-15 21:06:17,2017-10-01,2017-08-15 10:17:55,2017-08-01,7,2.0,11,13.0,0.0,0.043221
6,C_ID_7e63323c00,0.230129,2016-12-01,2018-02-23 16:09:32,2018-03-01,2017-01-05 12:19:24,2017-01-01,2018-04-06 17:52:25,2018-05-01,2018-03-06 12:37:14,2018-03-01,14,2.0,15,17.0,0.0,0.000891
7,C_ID_dfa21fc124,2.135850,2017-09-01,2017-12-21 19:44:42,2018-01-01,2017-09-28 21:24:46,2017-09-01,2018-02-18 20:27:17,2018-03-01,2018-02-06 07:03:41,2018-02-01,4,1.0,4,6.0,1.0,0.001918
8,C_ID_fe0fdac8ea,-0.065406,2017-08-01,2017-12-09 13:02:59,2018-01-01,2017-08-18 16:21:38,2017-08-01,2018-03-20 21:23:02,2018-04-01,2018-03-17 14:22:31,2018-03-01,5,1.0,5,8.0,0.0,0.007651
9,C_ID_bf62c0b49d,0.300062,2016-08-01,2018-02-28 16:49:37,2018-03-01,2017-01-09 15:59:16,2017-01-01,2018-04-08 13:53:01,2018-05-01,2018-04-01 09:22:06,2018-04-01,14,1.0,19,18.0,0.0,0.001339
